# Practica 2 -  Clasificación supervisada en scikit-learn

## Mineria de Datos 2017/2018

* **Hernan Indibil de La Cruz Calvo**
* **Alejandro Martin Simon Sanchez**

## Indice
1. Introduccion
2. Clasificadores y metodos de evaluacion


## 1. Introduccion
Esta práctica tendrá dos partes:

Primero estudiaremos la API de algunos de los clasificadores más utilizados en `scikit-learn` para conocer los distintos hiperparámetros que los configuran y estudiar los modelos resultantes.

Segundo estudiaremos métodos de selección de modelos, orientados a obtener una configuración óptima de los hiperparámetros para nuestros clasificadores.

In [1]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [2]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [3]:
# Se establece una semilla predeterminada para que los experimentos sean reproducibles
seed = 6470
np.random.seed(seed)

* **Lo siguiente es cargar los datos que se van a utilizar.**

    Se usa como label la variable categórica.

In [4]:
# Diccionario de nombre: fichero, con los datos de los dataframe a cargar
files = {
    'pima': '../data/pima.csv',
    'wisconsin': '../data/wisconsin.csv'
}

In [5]:
# Se cargan los dataframes
dfs = {name: pd.read_csv(file, dtype={ "label": 'category'}) for name, file in files.items()}

Como vimos en la práctica anterior, las variables de Pima "plas", "pres", "skin", "insu" y "mass" tienen los valores perdidos codificados como 0, ya que es imposible que una persona viva tenga valor 0 en cualquiera de ellas.
Podemos cambiar los 0 por NaN en el dataframe original sin perder información ni sobreajustar de ninguna forma, ya que no usamos información del conjunto de datos.

In [6]:
dfs['pima']['plas'] = dfs['pima']['plas'].replace(0, np.nan)
dfs['pima']['pres'] = dfs['pima']['pres'].replace(0, np.nan)
dfs['pima']['skin'] = dfs['pima']['skin'].replace(0, np.nan)
dfs['pima']['insu'] = dfs['pima']['insu'].replace(0, np.nan)
dfs['pima']['mass'] = dfs['pima']['mass'].replace(0, np.nan)

Para el dataframe Wisconsin la variable Patient ID no aporta nada beneficioso al proceso de clasificación, sólo sobreajuste. No tiene nada que ver con la variable clase. Por ello, procedemos a eliminarla.

In [7]:
dfs['wisconsin'] = dfs['wisconsin'].drop('patientId', 1)

Ahora procedemos a actualizar el diccionario de dataframes para que tenga la siguiente estructura:

* dfs
    * pima
        * train
            * atts
            * label
        * test
            * atts
            * label
    * wisconsin
        * train
            * atts
            * label
        * test
            * atts
            * label

La función utilizada no solo crea la estructura para los dataframes pima y wisconsin, sino para todos los dataframes que haya en el diccionario que se le pase. Realiza el proceso de holdout también para todos los dataframes.

In [8]:
from sklearn.model_selection import train_test_split

def holdoutAll(dframes, seed, tsize = 0.2):
    for k in dframes:
        dfAttributes = dframes[k].drop('label', 1)
        dfLabel = dframes[k]['label']

        dfs = {}
        dfs[k] = {}
        dfs[k]['train'] = {}
        dfs[k]['test'] = {}

        dfs[k]['train']['atts'], dfs[k]['test']['atts'], dfs[k]['train']['label'], dfs[k]['test']['label'] = train_test_split(
            dfAttributes,
            dfLabel,
            test_size = tsize,
            random_state = seed,
            stratify = dfLabel)

dfs = holdoutAll(dfs, seed, 0.2)

## 2. Clasificadores y metodos de evaluacion

### 2.1 KNN

KNN es un clasificador sencillo de entender y configurar, ya que solo tendremos que fijar el parámetro `k` que determina el número de vecinos con los que compararemos.

Se trata de un algoritmo perezoso, es decir, que no realiza fase de aprendizaje previa porque computa los parámetros necesarios para la clasificación durante el propio proceso de clasificación. Aunque esto pueda parecer una ventaja puede llegar a resultar demasiado pero ineficiente para datos muy grandes. Además, es muy sensible a cambios en los datos de training. 

In [9]:
from sklearn import neighbors

In [10]:
# Se prueba el clasificador con un valor de k = 5 vecinos.
k = 5
model = neighbors.KNeighborsClassifier(k)
# Como es perezoso solo se inicializa su estado.

# A continuacion se aprenden los datos del conjunto de training.
knn = model.fit(train_atts, train_label)

prediction = knn.predict(test_atts)

NameError: name 'train_atts' is not defined

In [ ]:
from sklearn.preprocessing import Imputer

In [ ]:
imp = Imputer(missing_values = '0', strategy = 'mean', axis = 0)
imp.fit(dfs[pima])

In [ ]:
from sklearn.model_selection import GridSearchCV